# Voice Recognition

Kali ini saya ingin membuat model sederhana yang bisa mengenali suara atau Voice Recognition. Terdiri dari 2 label yaitu buka dan tutup.

## Pengambilan Data

Untuk dataset, saya membuat data sendiri dengan cara merekam suara pengucapan kata buka atau tutup. Dataset yang saya buat terdiri dari file dengan pengucapan kata buka sebanyak 100 file dan pengucapan kata tutup sebanyak 100 file yang tentunya dengan nada yang bervariasi. Saya taruh kedua suara tersebut secara terpisah kedalam folder yang terpisah juga. Saya menggunakan website https://online-voice-recorder.com/ .

### a. Pengubahan Nama File Serentak

Untuk mempermudah proses ekstraksi audio, saya ubah nama file dari sebelumnya Record (online-voice-recorder.com).mp3 menjadi buka(1-100).mp3 dan tutup(1-100).mp3 menggunakan code:

```{code}
import os

folder_path = r"{path}\voice\tutup"

for i, filename in enumerate(os.listdir(folder_path), start=1):
    if filename.split(".")[-1] == "mp3":
        old_path = os.path.join(folder_path, filename)
        if os.path.isfile(old_path):
            filename, file_extension = os.path.splitext(filename)
            new_name = f"tutup{i}.mp3"
            new_path = os.path.join(folder_path, new_name)
            os.rename(old_path, new_path)

# print("Semua file berhasil diubah namanya.")
```

![Teks alternatif](img/Screenshot%202025-10-27%20131651.png)

### b. Ekstraksi Audio menjadi CSV

Pertama kita ekstrak audio menjadi data dengan beberapa fitur. Untuk ekstraksi audio saya menggunakan library liborsa. Code dibawah akan mengekstrak audio dan otomatis akan save file berbentuk CSV. Namun untuk data suara buka dan tutup masih terdapat pada file CSV terpisah.

```{code}
import librosa
import tsfel
import pandas as pd
import os

var = "tutup"
folder_path = f"./voice/{var}"

# Ambil semua file audio di folder
audio_files = [f for f in os.listdir(folder_path) if f.endswith(".mp3")]

all_features = []  # list untuk menampung fitur setiap file

# konfigurasi fitur TSFEL
cfg = tsfel.get_features_by_domain("statistical")

for file_name in audio_files:
    file_path = os.path.join(folder_path, file_name)

    # load audio
    signal, sr = librosa.load(file_path, sr=None)

    # ekstraksi fitur
    X = tsfel.time_series_features_extractor(cfg, signal, fs=sr)

    # tambahkan info nama file
    X["label"] = var

    # simpan ke list
    all_features.append(X)

    print(f"Fitur diekstrak dari {file_name} — shape: {X.shape}")

# gabungkan semua ke satu DataFrame
final_df = pd.concat(all_features, ignore_index=True)

# simpan ke satu csv
output_path = f"./hasil/{var}.csv"
final_df.to_csv(output_path, index=False)

print(f"\nAll features saved successfully to: {output_path}")
print("Final shape:", final_df.shape)

df = pd.read_csv(f"./hasil/{var}.csv")
print(df.head())
print(df.info())
print(df.columns)
```

Didalam CSV akan terdapat 31 fitur hasil ekstraksi dan satu fitur berupa label.

```{code}
output/terminal

Final shape: (4, 32)
   0_Absolute energy  0_Average power  0_ECDF Percentile Count_0  0_ECDF Percentile Count_1  ...  0_Skewness  0_Standard deviation  0_Variance  label
0         781.998361       458.924837                    16358.0                    65433.0  ...    0.270414              0.097779    0.009561  tutup
1        2950.804306      1413.234554                    20044.0                    80179.0  ...    0.016671              0.171587    0.029442  tutup
2         690.249237       355.070285                    18662.0                    74649.0  ...    0.183912              0.086007    0.007397  tutup
3         615.859208       427.686193                    13824.0                    55296.0  ...    0.210164              0.094393    0.008910  tutup

[4 rows x 32 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   0_Absolute energy            4 non-null      float64
 1   0_Average power              4 non-null      float64
 2   0_ECDF Percentile Count_0    4 non-null      float64
 3   0_ECDF Percentile Count_1    4 non-null      float64
 4   0_ECDF Percentile_0          4 non-null      float64
 5   0_ECDF Percentile_1          4 non-null      float64
 6   0_ECDF_0                     4 non-null      float64
 7   0_ECDF_1                     4 non-null      float64
 8   0_ECDF_2                     4 non-null      float64
 9   0_ECDF_3                     4 non-null      float64
 10  0_ECDF_4                     4 non-null      float64
 11  0_ECDF_5                     4 non-null      float64
 12  0_ECDF_6                     4 non-null      float64
 13  0_ECDF_7                     4 non-null      float64
 14  0_ECDF_8                     4 non-null      float64
 15  0_ECDF_9                     4 non-null      float64
 16  0_Entropy                    4 non-null      float64
 17  0_Histogram mode             4 non-null      float64
 18  0_Interquartile range        4 non-null      float64
 19  0_Kurtosis                   4 non-null      float64
 20  0_Max                        4 non-null      float64
 21  0_Mean                       4 non-null      float64
 22  0_Mean absolute deviation    4 non-null      float64
 23  0_Median                     4 non-null      float64
 24  0_Median absolute deviation  4 non-null      float64
 25  0_Min                        4 non-null      float64
 26  0_Peak to peak distance      4 non-null      float64
 27  0_Root mean square           4 non-null      float64
 28  0_Skewness                   4 non-null      float64
 29  0_Standard deviation         4 non-null      float64
 30  0_Variance                   4 non-null      float64
 31  label                        4 non-null      object
dtypes: float64(31), object(1)
```

Lalu saya menggabungkan kedua CSV.

```{code}
import pandas as pd

df_buka = pd.read_csv("./hasil/buka.csv")
df_tutup = pd.read_csv("./hasil/tutup.csv")

df_total = pd.concat([df_buka, df_tutup], ignore_index=True)

df_total.to_csv("./hasil/voice.csv", index=False)

print("File berhasil digabung! Bentuk data:", df_total.shape)
```

## Preproccessing Data

Setelah mendapatkan 1 file CSV utuh yang terdapat data suara buka dan tutup. Sekarang kita akan preproccessing data seperti mengecek missing value, mendeteksi outlier, dll.

### Missing Value

```{code}
import pandas as pd
df = pd.read_csv("./hasil/voice.csv")
print(df.info())

output/terminal

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   0_Absolute energy            9 non-null      float64
 1   0_Average power              9 non-null      float64
 2   0_ECDF Percentile Count_0    9 non-null      float64
 3   0_ECDF Percentile Count_1    9 non-null      float64
 4   0_ECDF Percentile_0          9 non-null      float64
 5   0_ECDF Percentile_1          9 non-null      float64
 6   0_ECDF_0                     9 non-null      float64
 7   0_ECDF_1                     9 non-null      float64
 8   0_ECDF_2                     9 non-null      float64
 9   0_ECDF_3                     9 non-null      float64
 10  0_ECDF_4                     9 non-null      float64
 11  0_ECDF_5                     9 non-null      float64
 12  0_ECDF_6                     9 non-null      float64
 13  0_ECDF_7                     9 non-null      float64
 14  0_ECDF_8                     9 non-null      float64
 15  0_ECDF_9                     9 non-null      float64
 16  0_Entropy                    9 non-null      float64
 17  0_Histogram mode             9 non-null      float64
 18  0_Interquartile range        9 non-null      float64
 19  0_Kurtosis                   9 non-null      float64
 20  0_Max                        9 non-null      float64
 21  0_Mean                       9 non-null      float64
 22  0_Mean absolute deviation    9 non-null      float64
 23  0_Median                     9 non-null      float64
 24  0_Median absolute deviation  9 non-null      float64
 25  0_Min                        9 non-null      float64
 26  0_Peak to peak distance      9 non-null      float64
 27  0_Root mean square           9 non-null      float64
 28  0_Skewness                   9 non-null      float64
 29  0_Standard deviation         9 non-null      float64
 30  0_Variance                   9 non-null      float64
 31  label                        9 non-null      object
dtypes: float64(31), object(1)
memory usage: 2.4+ KB
```

Berdasarkan hasil diatas, tidak terdapat nilai null atau Missing Value.

## Modeling

Setelah kita cek Missing Value, tahap selanjutnya adalah train model. Disini kita menggunakan model K Nearest Neighbor (KNN).

```{code}
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import tsfel
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score
)

# === 1. Baca data hasil ekstraksi ===
df = pd.read_csv("hasil/voice.csv")

# === 2. Pilih kolom X dan y ===
X = df.drop(columns=['label', 'file_name'])
y = df['label']

# === 3. Encode label menjadi numerik ===
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # contoh: buka -> 0, tutup -> 1

# === 4. Normalisasi fitur ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === 5. Split data ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# === 6. Train model KNN ===
k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# === 7. Prediksi ===
y_pred = knn.predict(X_test)

# === 8. Evaluasi dasar ===
print("=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=0, zero_division=0)  # 0 = buka
recall = recall_score(y_test, y_pred, pos_label=0, zero_division=0)
f1 = f1_score(y_test, y_pred, pos_label=0, zero_division=0)

print(f"Akurasi  : {accuracy:.3f}")
print(f"Presisi  : {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1-Score : {f1:.3f}")
print(X_train.shape)

# === 9. Confusion Matrix ===
plt.figure(figsize=(5,4))
sns.heatmap(
    confusion_matrix(y_test, y_pred),
    annot=True, cmap="Blues", fmt='d',
    xticklabels=le.classes_,
    yticklabels=le.classes_
)
plt.title("Confusion Matrix KNN (Suara Buka/Tutup)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# === 9. Simpan model, scaler dan feature names ===
pickle.dump(knn, open("./voice_recognition/model_knn.pkl", "wb"))
pickle.dump(scaler, open("./voice_recognition/scaler.pkl", "wb"))
pickle.dump(list(X.columns), open("./voice_recognition/feature_names.pkl", "wb"))

print("✅ Model, scaler dan feature names berhasil disimpan!")
```

Code diatas untuk melatih model KNN dan menyimpan model, scaler dan feature names dalam bentuk .pkl.

```{code}
output/terminal
=== Classification Report ===
              precision    recall  f1-score   support

        buka       0.78      0.70      0.74        20
       tutup       0.73      0.80      0.76        20

    accuracy                           0.75        40
   macro avg       0.75      0.75      0.75        40
weighted avg       0.75      0.75      0.75        40

Akurasi  : 0.750
Presisi  : 0.778
Recall   : 0.700
F1-Score : 0.737
(160, 31)
```

![Teks alternatif](img/Screenshot%202025-11-02%20154801.png)

Gambar diatas adalah Confusion Matrix untuk melihat prediksi dari model KNN. Sumbu X menyatakan hasil prediksi dari model KNN dan sumbu Y menyatakan label sebenarnya.

Tips abal-abal untuk menyaring kualitas data. Setelah kita menyimpan model dalam bentuk .pkl, setelah itu kita langsung prediksi dengan mengambil 1 suara sampel:

```{code}
# kita tes data buka
var = "buka"

# load file .pkl
model = pickle.load(open("./voice_recognition/model_knn.pkl", "rb"))
scaler = pickle.load(open("./voice_recognition/scaler.pkl", "rb"))
feature_names = pickle.load(open("./voice_recognition/feature_names.pkl", "rb"))

cfg = tsfel.get_features_by_domain("statistical")
# signal, sr = librosa.load("path suara sampel", sr=None)
var = "buka"
for i in range(1, 21):
    signal, sr = librosa.load(f"./voice/{var}/{var}{i}.mp3", sr=None)
    X = tsfel.time_series_features_extractor(cfg, signal, fs=sr)

    X_scaled = scaler.transform(X)
    y_pred_new = model.predict(X_scaled)[0]
    pred_label = le.inverse_transform([y_pred_new])[0]
    print(f"🎯 Hasil prediksi: {pred_label} - {var}{i}.mp3")
```

```{code}
output/terminal
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka1.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka2.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka3.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka4.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka5.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka6.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka7.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka8.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka9.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka10.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka11.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka12.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka13.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka14.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka15.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka16.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka17.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka18.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: tutup - buka19.mp3
Progress: |██████████████████████████████████████████████████| 100% Complete
🎯 Hasil prediksi: buka - buka20.mp3
```

Setelah itu kita identifikasi prediksi yang salah, lalu hapus file mp3 nya dan record ulang.

## Deployment

Untuk deploy model kita gunakan Streamlit Python.

```{code}
pip install streamlit
```

```{code}
import streamlit as st
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write
import tempfile
import tsfel
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

# === LOAD MODEL DAN SCALER ===
model = pickle.load(open("model_knn.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))
feature_names = pickle.load(open("feature_names.pkl", "rb"))

st.title("🔊 Klasifikasi Suara Buka / Tutup")
st.markdown("Rekam suaramu lalu biarkan model KNN menebak apakah itu **'buka'** atau **'tutup'**.")

# === STEP 1: Rekam Suara ===
duration = 3
if st.button("🎙️ Rekam Sekarang"):
    st.info("Merekam... silakan ucapkan kata 'buka' atau 'tutup'")
    fs = 44100  # sample rate
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    st.success("✅ Rekaman selesai!")

    # Simpan ke file sementara
    temp_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    write(temp_wav.name, fs, recording)

    st.audio(temp_wav.name, format="audio/mp3")

    # === STEP 2: Ekstraksi Fitur TSFEL ===
    st.write("⏳ Mengekstraksi fitur suara...")
    features = tsfel.time_series_features_extractor(
        tsfel.get_features_by_domain(),
        recording.flatten(),
        fs=fs
    )

    features = features.reindex(columns=feature_names, fill_value=0)

    # === STEP 3: Normalisasi & Prediksi ===
    label_map = {0: "buka", 1: "tutup"}
    X = scaler.transform(features)

    # --- 🔮 Prediksi dan Probabilitas ---
    prediction = model.predict(X)[0]
    probs = model.predict_proba(X)[0]  # <-- Tambahan

    epsilon = 0.05  # semakin besar, semakin lembut
    probs = (probs + epsilon) / (probs + epsilon).sum()

    label = label_map.get(prediction, "Tidak diketahui, silahkan rekam ulang")

    st.success(f"🎯 Hasil prediksi: **{label}**")

    # --- 📊 Tampilkan probabilitas ---
    prob_df = pd.DataFrame({
        "Label": ["buka", "tutup"],
        "Probabilitas": [probs[0], probs[1]]
    })
    # st.bar_chart(prob_df.set_index("Label"))

    st.write("### Nilai probabilitas:")
    st.write(f"🟢 **Buka:** {probs[0]*100:.2f}%")
    st.write(f"🔵 **Tutup:** {probs[1]*100:.2f}%")
```

Code diatas untuk menginstall Streamlit dan membuat aplikasi sederhana dari Streamlit beserta interface-nya. Dalam code diatas terdapat load model, scaler dan feature names dengan file berupa .pkl hasil dari menimpan model pada code sebelumnya. Berikut adalah tampilan interface sederhana:

![Teks alternatif](img/Screenshot%202025-11-02%20155231.png)

Lalu setelah itu, kita upload model kita ke cloud Streamlit

a. Pertama login ke Streamlit https://streamlit.io/ . Klik free di pojok kanan atas lalu pilih connect with GitHub.

![Teks alternatif](img/Screenshot%202025-11-02%20160818.png)

b. Push project ke GitHub dengan struktur folder sebagai berikut:

```{code}
voice_app/
│
├── model_knn.pkl           ← model yang sudah dilatih & disimpan
├── scaler.pkl              ← scaler (StandardScaler) jika kamu pakai normalisasi
├── feature_names.pkl       ← untuk menyimpan nama fitur dari model
├── app.py                  ← file utama streamlit
└── requirements.txt        ← (opsional) untuk deploy ke cloud
```

3. Setelah itu isi data sesuai dari repositori GitHub kalian.

4. Selesai, sekarang modelmu sudah di upload ke Cloud Streamlit seperti: https://voice-recognition-726v9k54uzp5j5dpezesfs.streamlit.app/ .

5. Code Streamlit saya:

```{code}
import streamlit as st
from audio_recorder_streamlit import audio_recorder
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import soundfile as sf
from pydub import AudioSegment
import joblib
import librosa
import os
import io
import tsfel

st.title("🎙️ Voice Classifier: Buka / Tutup")
st.markdown("Rekam suaramu lalu biarkan model KNN menebak apakah itu **'buka'** atau **'tutup'**.")

model_path = os.path.join(os.path.dirname(__file__), "model_knn.pkl")
scaler_path = os.path.join(os.path.dirname(__file__), "scaler.pkl")
feature_names_path = os.path.join(os.path.dirname(__file__), "feature_names.pkl")

model = joblib.load(model_path)
scaler = joblib.load(scaler_path)
feature_names = joblib.load(feature_names_path)

le = LabelEncoder()
y_encoded = le.fit_transform(["buka", "tutup"])

if "last_audio" not in st.session_state:
    st.session_state.last_audio = None

mode = "🎙️ Rekam langsung"

def analyze_audio(audio_bytes, source="rekaman"):
    try:
        audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes), format="wav")
        samples = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)

        if audio_segment.sample_width == 2:
            samples = samples / 32768.0
        elif audio_segment.sample_width == 4:
            samples = samples / 2147483648.0

        if audio_segment.channels == 2:
            samples = samples.reshape((-1, 2)).mean(axis=1)

        sr = audio_segment.frame_rate

        # Resample ke 16kHz jika perlu
        if sr != 16000:
            samples = librosa.resample(samples, orig_sr=sr, target_sr=16000)
            sr = 16000
        
        # Normalisasi
        samples = samples / (np.max(np.abs(samples)) + 1e-8)

        # Simpan untuk preview
        sf.write("temp_audio.wav", samples, sr)
        st.audio("temp_audio.wav", format="audio/wav")

        st.write("⏳ Mengekstraksi fitur...")
        cfg = tsfel.get_features_by_domain("statistical")
        features = tsfel.time_series_features_extractor(cfg, samples, fs=sr, verbose=0)

        # Pastikan kolom sama dengan feature_names
        features = features.reindex(columns=feature_names, fill_value=0)

        # === Prediksi ===
        label_map = {0: "buka", 1: "tutup"}
        X = scaler.transform(features)
        prediction = model.predict(X)[0]
        probs = model.predict_proba(X)[0]  # <-- Tambahan

        epsilon = 0.05  # semakin besar, semakin lembut
        probs = (probs + epsilon) / (probs + epsilon).sum()
        label = label_map.get(prediction, "Tidak diketahui, silahkan rekam ulang")

        st.success(f"🎯 Hasil prediksi: **{label}**")

        # --- 📊 Tampilkan probabilitas ---
        prob_df = pd.DataFrame({
            "Label": ["buka", "tutup"],
            "Probabilitas": [probs[0], probs[1]]
        })
        # st.bar_chart(prob_df.set_index("Label"))

        st.write("Nilai probabilitas:")
        st.write(f"🟢 **Buka:** {probs[0]*100:.2f}%")
        st.write(f"🔵 **Tutup:** {probs[1]*100:.2f}%")

    except Exception as e:
        st.error(f"❌ Error saat analisis: {e}")

if mode == "🎙️ Rekam langsung":
    st.info("🎙️ Tekan tombol mikrofon di bawah, ucapkan **'BUKA'** atau **'TUTUP'** dengan jelas, lalu tekan stop.")
    
    # Audio recorder widget
    audio_bytes = audio_recorder(
        text="Klik untuk merekam",
        recording_color="#e74c3c",
        neutral_color="#3498db",
        icon_name="microphone",
        icon_size="2x",
        pause_threshold=2.0,
        sample_rate=16000
    )
    
    # Jika ada audio baru yang direkam
    if audio_bytes:
        # Cek apakah ini audio baru (berbeda dari sebelumnya)
        if audio_bytes != st.session_state.last_audio:
            st.session_state.last_audio = audio_bytes
            
            st.success("✅ Audio berhasil direkam! Menganalisis...")
            
            # Analisis otomatis
            with st.spinner("🔄 Memproses audio..."):
                analyze_audio(audio_bytes, source="rekaman")
        else:
            # Audio sama dengan sebelumnya, tampilkan tombol analisis ulang
            st.info("ℹ️ Audio sudah dianalisis. Rekam ulang untuk prediksi baru.")
            
            if st.button("🔄 Analisis Ulang", type="secondary"):
                with st.spinner("🔄 Memproses audio..."):
                    analyze_audio(audio_bytes, source="rekaman")
    else:
        st.info("👆 Klik tombol mikrofon di atas untuk mulai merekam")

```

6. Isi requirements.txt

```{code}
streamlit
audio-recorder-streamlit
numpy
pandas
scikit-learn
joblib
librosa
soundfile
pydub
tsfel
```